In [23]:
import pandas as pd
import re

# Shows more characters in the output when you print data
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 300)

def format_by_transaction_code(dataframe):
    col_names=['Organisation','Transaction_Date','IsContactless','Extra_Info']
    formatted_dataframe = pd.DataFrame(columns=col_names)
    for index, code in enumerate(dataframe['Transaction Code']):
        match code:
            case 'POS':
                length = len(dataframe.loc[index,'Transaction_Description'].split(','))
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                if length > 2:
                    df[['Card_Date_Contactless','Organisation','Extra_Info']] = df[0].str.split(',',2, expand=True)
                    if len(df['Card_Date_Contactless'].str.split()[0]) == 2:
                        df[['Card_Num','Transaction_Date']] = df['Card_Date_Contactless'].str.split(expand=True)
                    else:
                        df[['Card_Num','Transaction_Date','IsContactless']] = df['Card_Date_Contactless'].str.split(expand=True)
                    df.drop(columns=[0], inplace=True)
                    df.drop(columns=['Card_Date_Contactless'], inplace=True)
                    df.drop(columns=['Card_Num'], inplace=True)
                else:
                    df[['Card_Date_Contactless','Organisation']] = df[0].str.split(',',1, expand=True)
                    if len(df['Card_Date_Contactless'].str.split()[0]) == 2:
                        df[['Card_Num','Transaction_Date']] = df['Card_Date_Contactless'].str.split(expand=True)
                    else:
                        df[['Card_Num','Transaction_Date','IsContactless']] = df['Card_Date_Contactless'].str.split(expand=True)
                    df.drop(columns=[0], inplace=True)
                    df.drop(columns=['Card_Date_Contactless'], inplace=True)
                    df.drop(columns=['Card_Num'], inplace=True)
                # Add to master Dataframe
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case 'BAC':
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                df['Organisation'] = df[0].str.split(',')[0][0]
                df.drop(columns=[0], inplace=True)
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case 'DPC':
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                df.rename(columns={0:'Extra_Info'}, inplace=True)
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case 'D/D':
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                df['Organisation'] = df[0].str.split(',')[0][0]
                df.drop(columns=[0], inplace=True)
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case 'CDM':
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                df.rename(columns={0:'Extra_Info'}, inplace=True)
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case _:
                # Note: We need to do something to handle unknown transaction types
                print("Sorry we cant handle this transaction type yet.")
    return formatted_dataframe

def format_data(user_id):
    # Reads in the Transaction and Balance Json files from Nordigen, Drops the Transaction ID column since it has
    # useless information
    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\{user_id}_Transactions.json")

    # Pending transactions does not sync with the balance in Nordigen, as a result we are adding the pending
    # transactions to our booked transactions to compensate and show the proper balance.
    df_transactions = pd.DataFrame(df['transactions']['booked'])
    df_transactions_pending = pd.DataFrame(df['transactions']['pending'])

    df_transactions = pd.concat([df_transactions, df_transactions_pending])
    df_transactions["bookingDate"] = pd.to_datetime(df_transactions["bookingDate"])
    df_transactions.reset_index(inplace=True, drop=True)

    df_transactionamount = pd.json_normalize(df_transactions['transactionAmount'])
    df_transactions = pd.concat([df_transactions, df_transactionamount], axis=1).drop('transactionAmount', axis = 1)
    df_transactions.drop(columns=['transactionId'], inplace=True)
    df_transactions = df_transactions.sort_values(by='bookingDate', ascending=False)
    df_transactions.reset_index(inplace=True, drop=True)

    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\Testid_00000009_Balances.json")
    # Since we will use booked transactions from the "Transactions" data, we only want to see the forwardavailable
    # balance from the "Balance" data
    df_balance = pd.DataFrame(df['balances'][1])

    # This creates a balance column using the current balance in the "balance" data taken from the Nordigen API
    current_balance = []
    value = float(df_balance['balanceAmount'].loc['amount'])
    for balance in df_transactions['amount']:
        current_balance.append(value)
        value -= float(balance)
    df_transactions['balance'] = current_balance

    # This removed all the unexpected whitespace, that is longer than expected, reducing them to a specified size, '  ',
    # this space is then replaced by ',' so that it can be separated by the comma later when creating new columns.
    finished = False
    while not finished:
        counter = 0
        for index, description in enumerate(df_transactions['remittanceInformationUnstructured']):
            Tokens = re.findall('\s+', description)
            for token in Tokens:
                if len(token) > len('  '):
                    df_transactions.loc[index,'remittanceInformationUnstructured'] = df_transactions.loc[index,'remittanceInformationUnstructured'].replace(' '*len(token),'  ')
                    counter += 1
        if counter == 0:
            finished = True
            print("Finished!")
        else:
            print("Needs another go!")
    df_transactions['remittanceInformationUnstructured'] = df_transactions['remittanceInformationUnstructured'].str.replace('  ',',').str.strip()

    df = pd.read_csv('Transaction Explanations.txt',sep = ';')
    df_transactions.rename(columns = {'proprietaryBankTransactionCode':'Transaction Code'}, inplace = True)
    df_transactions.rename(columns = {'remittanceInformationUnstructured':'Transaction_Description'}, inplace = True)
    df_transactions = df_transactions.merge(df, on ='Transaction Code', how='left')
    df_transaction_descriptions = format_by_transaction_code(df_transactions)

    df_transactions.drop(columns=['Transaction_Description'], inplace=True)
    df_transactions = pd.concat([df_transactions, df_transaction_descriptions], axis=1)
    return df_transactions
df = format_data("Testid_00000011")
df



Needs another go!
Needs another go!
Needs another go!
Finished!


,bookingDate,Transaction Code,amount,currency,balance,Code_Description,Explanation,Organisation,Transaction_Date,IsContactless,Extra_Info
0,2021-11-30,BAC,1399.11,GBP,2703.41,Automated Credit,Any payment into your account that is not cash or cheque,JCW SEARCH LIMITED,NaN,NaN,NaN
1,2021-11-29,POS,-9.60,GBP,1304.30,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,TFL TRAVEL CH,26NOV21,C,TFL.GOV.UK/CP GB
2,2021-11-29,POS,-6.00,GBP,1313.90,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,ZTL*HOSHI,26NOV21,C,LONDON GB
3,2021-11-19,POS,-8.00,GBP,1319.90,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,TFL TRAVEL CH,18NOV21,C,TFL.GOV.UK/CP GB
4,2021-11-19,POS,-4.90,GBP,1327.90,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,BEIGEL BAKE,18NOV21,C,LONDON GB
5,2021-11-17,POS,-8.80,GBP,1332.80,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,TFL TRAVEL CH,16NOV21,C,TFL.GOV.UK/CP GB
6,2021-11-15,POS,-1.55,GBP,1341.60,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,TFL TRAVEL CH,12NOV21,C,TFL.GOV.UK/CP GB
7,2021-11-15,POS,-5.20,GBP,1343.15,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,TFL TRAVEL CH,12NOV21,C,TFL.GOV.UK/CP GB
8,2021-11-15,POS,-6.62,GBP,1348.35,Maestro Transaction,Maestro is a brand of debit card issued by Mastercard. Maestro cards can be used for in-store pa...,HRK,13NOV21,NaN,TALLINN EE
9,2021-11-15,DPC,-33.00,GBP,1354.97,Digital Banking Payment,The transfer of money between two account holders.,NaN,NaN,NaN,"JACK,JACK,VIA ONLINE - PYMT FP 15/11/21 10,30002132470780000N"
